# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [4]:
# Your code here
q = """
SELECT firstName,lastName
FROM employees
JOIN offices
USING (officeCode)
WHERE city = 'Boston'
;
"""
pd.read_sql(q, conn)


,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [5]:
# Your code here
q = """
SELECT
    o.officeCode,
    o.city,
    COUNT(e.employeeNumber) AS n_employees
FROM offices AS o
LEFT JOIN employees AS e
    USING(officeCode)
GROUP BY officeCode
HAVING n_employees = 0
;
"""
pd.read_sql(q, conn)

,officeCode,city,n_employees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [6]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""


,city,country,total_sales
0,Paris,France,3083761.58
1,London,UK,1436950.70
2,San Francisco,USA,1429063.57
3,NYC,USA,1157589.72
4,Sydney,Australia,1147176.35
5,Boston,USA,892538.62
6,Tokyo,Japan,457110.07


In [7]:
"""
Question 1

"""

# Your code here
"""
What are the total sales (in USD) for each office?
"""
#answer
q = """
SELECT 
    o.city,
    o.country,
    ROUND(SUM(od.quantityOrdered * od.priceEach), 2) AS total_sales
FROM offices o
JOIN employees e USING(officeCode)
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders ord USING(customerNumber)
JOIN orderdetails od USING(orderNumber)
GROUP BY o.officeCode, o.city, o.country
ORDER BY total_sales DESC
;
"""
pd.read_sql(q,conn)


,city,country,total_sales
0,Paris,France,3083761.58
1,London,UK,1436950.70
2,San Francisco,USA,1429063.57
3,NYC,USA,1157589.72
4,Sydney,Australia,1147176.35
5,Boston,USA,892538.62
6,Tokyo,Japan,457110.07


In [8]:
"""
Question 2
"""


# Your code here
"""
Which customers have never placed an order?
"""

q = """
SELECT 
    c.customerNumber,
    c.customerName,
    c.contactFirstName,
    c.contactLastName
FROM customers c
LEFT JOIN orders o USING(customerNumber)
WHERE o.orderNumber IS NULL
ORDER BY c.customerName;
"""
pd.read_sql(q, conn)


,customerNumber,customerName,contactFirstName,contactLastName
0,237,ANG Resellers,Alejandra,Camino
1,168,American Souvenirs Inc,Keith,Franco
2,465,"Anton Designs, Ltd.",Carmen,Anton
3,206,"Asian Shopping Network, Co",Brydey,Walker
4,348,"Asian Treasures, Inc.",Patricia,McKenna
5,293,BG&E Collectables,Ed,Harrison
6,335,"Cramer Spezialitäten, Ltd",Philip,Cramer
7,307,Der Hund Imports,Mel,Andersen
8,443,"Feuer Online Stores, Inc",Alexander,Feuer
9,273,"Franken Gifts, Co",Peter,Franken


In [10]:
"""
Question 3
"""

# Your code here
"""
What is the average order value for each product line?
"""

q = """
SELECT 
    p.productLine,
    ROUND(AVG(od.quantityOrdered * od.priceEach), 2) AS avg_order_value,
    COUNT(DISTINCT od.orderNumber) AS total_orders
FROM products p
JOIN orderdetails od USING(productCode)
GROUP BY p.productLine
ORDER BY avg_order_value DESC;
"""
pd.read_sql(q,conn)

,productLine,avg_order_value,total_orders
0,Classic Cars,3815.76,209
1,Trucks and Buses,3325.04,75
2,Motorcycles,3123.75,79
3,Planes,2841.18,66
4,Vintage Cars,2736.01,187
5,Ships,2710.20,68
6,Trains,2327.57,47


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [11]:
# Your code here
q = """
SELECT firstName, lastName, productName
FROM employees AS e
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
JOIN products
    USING(productCode)
;
"""
df = pd.read_sql(q, conn)
df

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [12]:
# Your code here
q = """
SELECT firstName, lastName, SUM(quantityOrdered) as total_products_sold
FROM employees AS e
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY firstName, lastName
ORDER BY lastName
;
"""
pd.read_sql(q, conn)

,firstName,lastName,total_products_sold
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [13]:
# Your code here
q = """
SELECT firstName, lastName, COUNT(productCode) as different_products_sold
FROM employees AS e
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY firstName, lastName
HAVING different_products_sold > 200
ORDER BY lastName
;
"""
pd.read_sql(q, conn)

,firstName,lastName,different_products_sold
0,Larry,Bott,236
1,Pamela,Castillo,272
2,Gerard,Hernandez,396
3,Leslie,Jennings,331
4,Barry,Jones,220
5,George,Vanauf,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!